# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Loading the cities csv file created in part 1.
weather_file = "../WeatherPy/output_data/cities.csv"
weather_file_df = pd.read_csv(weather_file)

# Displaying the first 5 rows of the dataframe.
weather_file_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,New Norfolk,-42.7826,147.0587,48.92,69,9,2.04,AU,1624153359
1,Rikitea,-23.1203,-134.9692,75.36,74,3,17.87,PF,1624153360
2,Port Hawkesbury,45.6169,-61.3485,62.01,91,90,5.01,CA,1624153361
3,Alyangula,-13.8483,136.4192,75.07,57,100,8.05,AU,1624153362
4,Puerto Ayora,-0.7393,-90.3518,73.54,88,81,8.99,EC,1624153362


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configuring the api key.
gmaps.configure(api_key=g_key)

In [4]:
# Setting the locations from the csv file. Also coverting them to float to avoid errors reading the data.
locations_info = weather_file_df[["Lat", "Lng"]].astype(float)

# Setting the weights part of the heatmap with the humidity data. Again, coverting to float.
weight = weather_file_df["Humidity"].astype(float)

# Checking the maximum humidity in the file to use it for the max_intensity part of the heatmap.
humidity_max = weather_file_df["Humidity"].max().astype(float)
humidity_max

100.0

In [5]:
# Configuring the google maps figure. 
# Also I have entered in a zoom level and coordinates so the map displays the area we are looking at.
fig = gmaps.figure(zoom_level=2, center=(40,90))

# Creating the heat layer to the map with all the specifications defined.
heat_layer = gmaps.heatmap_layer(locations_info, weights=weight, dissipating=False, 
                                 max_intensity=humidity_max, point_radius=2)

# Adding the heat layer to the figure.
fig.add_layer(heat_layer)

# Displaying the figure.
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [6]:
# Creating a new dataframe based on the ideal weather conditions defined. 
# Here we use the .loc method on the columns we want to define parameters on.
ideal_weather_df = weather_file_df.loc[((weather_file_df["Max Temp"] > 75) & (weather_file_df["Max Temp"] < 85) &
                                  ((weather_file_df["Cloudiness"] < 40) & (weather_file_df["Cloudiness"] > 15)) &
                                   (weather_file_df["Wind Speed"] < 10) & (weather_file_df["Humidity"] < 75))]

# Next we drop any rows with null values
ideal_weather_df.dropna()

# Displaying the new dataframe.
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
22,Kumul,42.8000,93.4500,84.11,15,22,5.66,CN,1624153375
168,Lorengau,-2.0226,147.2712,84.76,68,29,5.73,PG,1624153482
201,Deputatsky,69.3000,139.9000,78.12,23,19,9.80,RU,1624153509
278,Marabu,4.4148,114.0089,82.54,69,20,6.91,MY,1624153565
478,São Miguel do Araguaia,-13.2750,-50.1628,77.05,44,21,3.91,BR,1624153710
501,Soe,-9.8607,124.2840,75.25,63,35,9.06,ID,1624153726


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Next we store relevant columns into the hotel_df variable.
hotel_df = ideal_weather_df.loc[:, ["City", "Lat", "Lng", "Country"]]

# Then we add a blank column to the dataframe.
hotel_df["Hotel Name"] = ""

# Display the df.
hotel_df

,City,Lat,Lng,Country,Hotel Name
22,Kumul,42.8000,93.4500,CN,
168,Lorengau,-2.0226,147.2712,PG,
201,Deputatsky,69.3000,139.9000,RU,
278,Marabu,4.4148,114.0089,MY,
478,São Miguel do Araguaia,-13.2750,-50.1628,BR,
501,Soe,-9.8607,124.2840,ID,


In [8]:
# In this section we will use the google maps API and get the closest hotel to each of the cities in our hotel_df.

# First we define our parameters.
params = {"radius": 5000, "types": "lodging", "key": g_key}

# Next we are going to go through each row of the hotel_df using the iterrows() method.
# Using the lat and lng data we are going to add location details to the parameters dictionary.
# We are then going to get the data required using the google maps url and applying our parameters to the search.
# Each row is looked up and the name of the closest hotel is added to the hotel_df. 
# If no hotel is found with the parameters the code will print "No hotel found.." and will skip that city.

for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat}, {lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_detail = requests.get(base_url, params=params)
    
    hotel_detail = hotel_detail.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_detail["results"][0]["name"]
            
    except:
        print(f"No hotel found within these parameteres for {city_name}... skipping") 
    

In [9]:
# Displaying the hotel_df to check the data.
hotel_df

,City,Lat,Lng,Country,Hotel Name
22,Kumul,42.8000,93.4500,CN,Laoyutang Resort
168,Lorengau,-2.0226,147.2712,PG,Lorengau Harbourside Hotel
201,Deputatsky,69.3000,139.9000,RU,Baza Otdykha
278,Marabu,4.4148,114.0089,MY,"Mega Hotel, Miri"
478,São Miguel do Araguaia,-13.2750,-50.1628,BR,Hotel Executivo Palace
501,Soe,-9.8607,124.2840,ID,Dena Hotel


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))